In [2]:
import sys
path = '/Users/Everwitt/Documents/Pitch-Prediction/pitcherprediction'
sys.path.append(path)

In [3]:
import pandas as pd
import numpy as np
from pitch_zone_config import generate_pitches

In [4]:
p = pd.read_csv('./all_pitches.csv')

In [5]:
pitches = generate_pitches()

In [6]:
p.head()

,px,py,code,type,pitch_type,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b,swing,res,zone
0,0.416,0.364,C,S,FF,2015000001,0,0,0,1,0.0,0.0,0.0,0,strike,2a
1,-0.191,-0.252,S,S,FF,2015000001,0,1,0,2,0.0,0.0,0.0,1,strike,4a
2,-0.518,0.685,F,S,FF,2015000001,0,2,0,3,0.0,0.0,0.0,1,foul,0a
3,-0.641,-1.378,B,B,FF,2015000001,0,2,0,4,0.0,0.0,0.0,0,ball,15b
4,-1.821,-0.516,B,B,CU,2015000001,1,2,0,5,0.0,0.0,0.0,0,ball,12b


In [7]:
p['zone'].value_counts()

12a    311062
15a    288733
13a    279727
4a     239050
7a     229461
8a     210612
3a     206808
5a     204035
6a     187835
1a     134066
15b    127702
0a     117847
16b    111816
13b    109396
12b    109055
2a     104105
10b     75499
16a     67266
10a     64783
14b     56338
9b      39108
14a     38355
11b     24377
9a       8575
-1        452
Name: zone, dtype: int64

In [8]:
import json, requests, urllib, io

In [12]:
# Pulling from swing data from GitHub
def pull_github_csv(user='mbendy', pao='ce9aec01691d2a8efde6a73dc3a6a5e337960199'):
    github_session = requests.Session()
    github_session.auth = (user, pao)

    # providing raw url to download csv from github
    url_swing = 'https://raw.githubusercontent.com/everwitt7/Pitch-Data/master/all_pitchers_rhh_swing_data.csv'
    url_take = 'https://raw.githubusercontent.com/everwitt7/Pitch-Data/master/all_pitchers_rhh_take_data.csv'

    download_s = github_session.get(url_swing).content
    df_swing = pd.read_csv(io.StringIO(download_s.decode('utf-8')), error_bad_lines=False)
    
    download_t = github_session.get(url_take).content
    df_take = pd.read_csv(io.StringIO(download_t.decode('utf-8')), error_bad_lines=False)
    
    df = df_swing.append(df_take, ignore_index=True)

    # pulling only necessary data  
    pitcher_actions = ["FF", "SL", "FT", "CH", "CU", "FC", "SI"]
    df = df[df['pitch_type'].isin(pitcher_actions)]
#     df_swing = df_swing[df_swing['swing'] == 1]

    return df

old_data = pull_github_csv()

In [16]:
print(old_data.shape)
print(p.shape) #interesting that p.shape is literally twice the size???
#actually this makes sense, we are looking ONLY at RHH

(1575773, 12)
(3346063, 16)


In [14]:
old_data.head()

,Unnamed: 0,ab_id,px,pz,code,pitch_type,b_count,s_count,zones,swing,result,batter_id
0,2,2015000003,-0.286,1.830,E,FF,2.0,0.0,6a,1,Hit,407812
1,3,2015000004,0.200,2.643,F,SI,0.0,0.0,4a,1,Foul,425509
2,4,2015000004,-0.741,2.930,F,FF,0.0,1.0,3a,1,Foul,425509
3,5,2015000004,-0.711,1.971,S,CU,0.0,2.0,6a,1,Strike,425509
4,6,2015000007,0.015,2.287,F,FF,0.0,0.0,4a,1,Foul,624585


In [15]:
old_data['zones'].value_counts() #hmmmm we actually see something similar!?

13a    164390
15a    132780
8a     119536
4a     118255
5a     113822
7a     109621
12a    109173
3a      83370
6a      70629
13b     67382
16b     66298
1a      64450
15b     55398
2a      53177
0a      49996
16a     39276
10b     33863
12b     32216
10a     28772
14b     20656
11b     14065
14a     13895
9b      11938
9a       2815
Name: zones, dtype: int64